In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import keras
import warnings

keras.utils.set_random_seed(42)
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',100)

df_train = pd.read_csv('./Code Ocean.csv')
df=df_train.copy()


In [8]:
#DATA CLEANING
#checking for duplicated values
df[df.duplicated()==True]

,term_months,home_ownership,purpose,addr_state,verification_status,application_type,loan_amnt,emp_length,annual_inc,dti,delinq_2yrs,revol_util,total_acc,credit_length_in_years,default_loan,int_rate,remain,issue_year,phi_term_month,phi_loan_amnt,phi_emp_length,phi_annual_inc,phi_dti,phi_delinq_2yrs,phi_revol_util,phi_total_acc,phi_credit_length_in_years,phi_int_rate,CRI,train_flag


In [9]:
#checking columns with a single unique value
for column in df.columns.tolist():
    if len(df[column].unique()) ==1:
        print(column)

train_flag


In [10]:
data = df.drop(['phi_term_month', 'phi_loan_amnt', 'phi_emp_length',
       'phi_annual_inc', 'phi_dti', 'phi_delinq_2yrs', 'phi_revol_util',
       'phi_total_acc', 'phi_credit_length_in_years', 'phi_int_rate','train_flag'],axis = 1)

In [11]:
#Now checking for outliers

from collections import Counter

def find_outliers(data,ftrs):
    """
    A python function to detect outliers. It takes in the dataframe data and checks if features ftrs have outliers
    """
    outlier_list = []

    # iterating over the features(ftr)
    for column in ftrs:

        # calculating interquartile range (Q3-Q1)
        Q1 = np.percentile(df[column], 25)
        Q3 = np.percentile(df[column],75)

        IQR = Q3 - Q1

        # outlier step
        outlier_step = 1.5 * IQR

        # Getting the indices of outliers
        outlier_indices_list = df[(df[column] < Q1 - outlier_step) | (df[column] > Q3 + outlier_step )].index.tolist()

        # adding the outlier_indices_list to the empty list
        outlier_list.extend(outlier_indices_list)

      # Checking for multiple outliers
      #multi_outliers = list( p for p, q in Counter(outlier_list).items() if q > n )

        return outlier_list

In [12]:
# calculating interquartile range (Q3-Q1)
Q1 = np.percentile(data['annual_inc'], 25)
Q3 = np.percentile(data['annual_inc'],75)

IQR = Q3 - Q1

# outlier step
outlier_step = 1.5 * IQR
data[(data['annual_inc'] < Q1 - outlier_step) | (data['annual_inc'] > Q3 + outlier_step )]

,term_months,home_ownership,purpose,addr_state,verification_status,application_type,loan_amnt,emp_length,annual_inc,dti,delinq_2yrs,revol_util,total_acc,credit_length_in_years,default_loan,int_rate,remain,issue_year,CRI
4,36,MORTGAGE,credit_card,MA,Verified,Joint App,8425,3,450000.0,12.37,0,65.7,37,21,0,27.27,-310.15,2018,72.718105
11,36,MORTGAGE,house,ID,Not Verified,Individual,15000,10,200000.0,2.62,0,39.2,45,12,0,7.56,-63.00,2018,55.556549
12,36,MORTGAGE,other,CA,Not Verified,Individual,8000,10,250000.0,20.1,0,61.2,29,31,0,7.56,-53.55,2018,71.136152
43,36,MORTGAGE,car,OK,Not Verified,Individual,25000,3,180000.0,11.77,0,24.5,32,14,0,7.02,-35.15,2018,56.460291
62,36,RENT,other,MA,Not Verified,Individual,20000,10,180000.0,18.62,0,28.4,30,16,0,7.02,-97.50,2018,65.103921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223617,36,OWN,other,CA,Verified,Individual,30000,10,560000.0,11.87,1,90.7,19,27,0,23.88,-3138.06,2017,100.996193
223624,36,MORTGAGE,car,MI,Not Verified,Individual,16000,10,175000.0,5.51,0,58.9,22,19,0,11.99,-693.26,2017,63.439307
223636,36,MORTGAGE,credit_card,CT,Verified,Individual,32000,3,750000.0,5.97,0,52.9,20,4,0,16.02,-4126.07,2017,61.767672
223674,36,RENT,debt_consolidation,IL,Verified,Individual,35000,2,275000.0,6.6,0,40,16,22,0,11.99,-318.66,2017,60.488572


In [13]:
#checking columns with a '' value
for column in data.columns.tolist():
    #if (dff[column] == np.NaN):
    data[column] = data[column].replace(r'^\s*$',np.nan,regex = True)
    print(column, data[column].unique())
    print('-'*50)

term_months [36 60]
--------------------------------------------------
home_ownership ['MORTGAGE' 'RENT' 'OWN' 'ANY' 'NONE']
--------------------------------------------------
purpose ['debt_consolidation' 'credit_card' 'other' 'house' 'car'
 'home_improvement' 'moving' 'small_business' 'vacation' 'medical'
 'major_purchase' 'renewable_energy' 'wedding']
--------------------------------------------------
addr_state ['CA' 'OH' 'WA' 'TX' 'MA' 'PA' 'MI' 'ID' 'AZ' 'NY' 'LA' 'RI' 'IL' 'TN'
 'NV' 'MN' 'NM' 'NJ' 'CO' 'VA' 'NC' 'FL' 'KY' 'OK' 'CT' 'MD' 'NH' 'IN'
 'GA' 'AL' 'MT' 'SC' 'DC' 'OR' 'NE' 'KS' 'HI' 'AK' 'VT' 'MO' 'UT' 'AR'
 'MS' 'WI' 'ND' 'DE' 'WV' 'ME' 'SD' 'WY']
--------------------------------------------------
verification_status ['Verified' 'Not Verified']
--------------------------------------------------
application_type ['Joint App' 'Individual']
--------------------------------------------------
loan_amnt [30000 40000 20000 ... 35450 37075 38125]
-----------------------------

In [14]:
#checking columns with a '' value
for column in data.columns.tolist():
    data[column] = data[column].replace(np.nan, data[column].mode()[0])
    print(column, data[column].unique())
    print('-'*50)

term_months [36 60]
--------------------------------------------------
home_ownership ['MORTGAGE' 'RENT' 'OWN' 'ANY' 'NONE']
--------------------------------------------------
purpose ['debt_consolidation' 'credit_card' 'other' 'house' 'car'
 'home_improvement' 'moving' 'small_business' 'vacation' 'medical'
 'major_purchase' 'renewable_energy' 'wedding']
--------------------------------------------------
addr_state ['CA' 'OH' 'WA' 'TX' 'MA' 'PA' 'MI' 'ID' 'AZ' 'NY' 'LA' 'RI' 'IL' 'TN'
 'NV' 'MN' 'NM' 'NJ' 'CO' 'VA' 'NC' 'FL' 'KY' 'OK' 'CT' 'MD' 'NH' 'IN'
 'GA' 'AL' 'MT' 'SC' 'DC' 'OR' 'NE' 'KS' 'HI' 'AK' 'VT' 'MO' 'UT' 'AR'
 'MS' 'WI' 'ND' 'DE' 'WV' 'ME' 'SD' 'WY']
--------------------------------------------------
verification_status ['Verified' 'Not Verified']
--------------------------------------------------
application_type ['Joint App' 'Individual']
--------------------------------------------------
loan_amnt [30000 40000 20000 ... 35450 37075 38125]
-----------------------------

In [15]:
data[['emp_length','dti','delinq_2yrs','revol_util','credit_length_in_years','remain','CRI']] = data[['emp_length','dti','delinq_2yrs','revol_util','credit_length_in_years','remain','CRI']].astype('float64')

In [16]:
# Import label encoder
from sklearn import preprocessing

# label_encoder object knows
# how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

# Encode labels in column 'species'.
data['home_ownership']= label_encoder.fit_transform(data['home_ownership'])
data['purpose']= label_encoder.fit_transform(data['purpose'])
data['addr_state']= label_encoder.fit_transform(data['addr_state'])
data['verification_status']= label_encoder.fit_transform(data['verification_status'])
data['application_type']= label_encoder.fit_transform(data['application_type'])

data['home_ownership'].unique(),data['purpose'].unique(),data['addr_state'].unique(),data['verification_status'].unique(),data['application_type'].unique()

(array([1, 4, 3, 0, 2]),
 array([ 2,  1,  8,  4,  0,  3,  7, 10, 11,  6,  5,  9, 12]),
 array([ 4, 34, 46, 42, 18, 37, 21, 12,  3, 33, 17, 38, 13, 41, 32, 22, 31,
        30,  5, 44, 26,  9, 16, 35,  6, 19, 29, 14, 10,  1, 25, 39,  7, 36,
        28, 15, 11,  0, 45, 23, 43,  2, 24, 47, 27,  8, 48, 20, 40, 49]),
 array([1, 0]),
 array([1, 0]))

In [17]:
train = data
#test = data[1100000:]

In [18]:
X = train.drop('default_loan',axis=1)#.values
#y = keras.utils.np_utils.to_categorical(train.default_loan.values)
y = train.default_loan

In [19]:
# Feature Scaling first to reduce training time
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_scaled = sc.fit_transform(X)

In [20]:
#spliting the X into train and test sets
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_scaled,y,train_size=0.8,random_state=42,shuffle=True,stratify=y)

In [21]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.optimizers import Adam,SGD,RMSprop

def create_mlp_model(input_dim):
    model = Sequential()
    model.add(Dense(units=128, input_dim=input_dim, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(units=64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(units=32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(units=1, activation='sigmoid'))#sigmoid and not relu should be the activation funtion of the output layer since we're expectioning 0s and 1s. ReLU is not bound to [0, 1] unlike Sigmoid
    return model

model = create_mlp_model(18)
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.1), metrics=['accuracy'])

In [24]:
import imblearn
from imblearn.combine import SMOTEENN
from collections import Counter

# Print class distribution before applying SMOTE-ENN
print("Class distribution before SMOTE-ENN:", Counter(y))

# Apply SMOTE-ENN
smote_enn = SMOTEENN(random_state=42)
X_resampled1, y_resampled1 = smote_enn.fit_resample(X[:100000], y[:100000])
X_resampled2, y_resampled2 = smote_enn.fit_resample(X[100000:200000], y[100000:200000])
X_resampled3, y_resampled3 = smote_enn.fit_resample(X[200000:300000], y[200000:300000])
#X_resampled4, y_resampled4 = smote_enn.fit_resample(X[300000:400000], y[300000:400000])
#X_resampled5, y_resampled5 = smote_enn.fit_resample(X[400000:500000], y[400000:500000])
#X_resampled6, y_resampled6 = smote_enn.fit_resample(X[500000:600000], y[500000:600000])
#X_resampled7, y_resampled7 = smote_enn.fit_resample(X[600000:700000], y[600000:700000])
#X_resampled8, y_resampled8 = smote_enn.fit_resample(X[700000:800000], y[700000:800000])
#X_resampled9, y_resampled9 = smote_enn.fit_resample(X[800000:900000], y[800000:900000])
#X_resampled10, y_resampled10 = smote_enn.fit_resample(X[900000:], y[900000:])
#X_resampled11, y_resampled11 = smote_enn.fit_resample(X[1000000:], y[1000000:])

Class distribution before SMOTE-ENN: Counter({0: 162761, 1: 60961})


In [25]:
X_resampled = pd.concat([X_resampled1,X_resampled2,X_resampled3],ignore_index=True)
y_resampled = pd.concat([y_resampled1,y_resampled2,y_resampled3],ignore_index=True)

# Print class distribution after applying SMOTE-ENN
#print("Class distribution after SMOTE-ENN:", Counter(y_resampled))

In [26]:
y_resampled_df=pd.DataFrame(y_resampled,columns=['default_loan'])
data_resampled = pd.concat([X_resampled, y_resampled_df],axis=1)

In [27]:
#spliting the X into train and test sets
#from sklearn.model_selection import train_test_split
X_train2,X_test2,y_train2,y_test2=train_test_split(X_resampled,y_resampled,train_size=0.8,random_state=42,shuffle=True,stratify=y_resampled)
#X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.8,random_state=7,shuffle=True,stratify=y)


In [28]:
from sklearn.metrics import *

In [29]:
from sklearn.metrics import roc_curve, auc

In [30]:
import timeit
import datetime

#mlp with resampling (ADAM optimizer)
# Start the timer
start_time = datetime.datetime.now()
# WOA hyperparameters
max_iter = 10  # Maximum number of iterations
pop_size = 10   # Population size
c1 = 2.0        # Constant for the spiral updating formula
c2 = 2.0        # Constant for the whale updating formula


# Initialize the population of models
population = [create_mlp_model(18) for _ in range(pop_size)]

loss_list = []

# Define the WOA algorithm
def whales_optimization_algorithm(X_train, y_train, population, max_iter, c1, c2):
    for iteration in range(max_iter):
        for i, model in enumerate(population):
            # Train the model with Adam optimizer
            model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.1), metrics=['accuracy'])
            history_woa = model.fit(X_train2, y_train2, validation_split=0.2, batch_size=512, epochs=1, verbose=True)

            # Calculate fitness based on accuracy
            y_pred = model.predict(X_test2)
            accuracy = accuracy_score(y_test2, (y_pred > 0.5).astype(int))

            # Update the model weights and biases using WOA
            a = 2.0 - iteration * ((2.0) / max_iter)  # Linearly decrease a from 2 to 0
            A = 2.0 * a * np.random.uniform() - a
            C = 2.0 * np.random.uniform()

            D = np.abs(C * np.array(model.get_weights()) - np.array(model.get_weights()))  # Difference between current weight and random whale
            new_weights = np.array(model.get_weights()) - A * D

            # Update the population with the improved model
            population[i].set_weights(new_weights)

        # Print the loss at the end of each iteration
        loss = model.evaluate(X_train2, y_train2, verbose=True)[0]
        loss_list.append(loss)
        print(f"Iteration {iteration + 1}/{max_iter}, Loss: {loss}")

# Apply the WOA algorithm
whales_optimization_algorithm(X_train2, y_train2, population, max_iter, c1, c2)

# Evaluate the best model from the population
best_model = max(population, key=lambda model: accuracy_score(y_test2, (model.predict(X_test2) > 0.5).astype(int)))

# Evaluate the best model on various metrics
y_pred_best_wao = best_model.predict(X_test2)
accuracy_wao = accuracy_score(y_test2, (y_pred_best_wao > 0.5).astype(int))
f1_wao = f1_score(y_test2, (y_pred_best_wao > 0.5).astype(int))
roc_auc_wao = roc_auc_score(y_test2, y_pred_best_wao)

print(f"Accuracy: {accuracy_wao}")
print(f"F1 Score: {f1_wao}")
print(f"AUC-ROC Score: {roc_auc_wao}")


# Stop the timer
end_time = datetime.datetime.now()

# Calculate the execution time
execution_time = (end_time - start_time).total_seconds()

print("Execution time: {:.2f} seconds".format(execution_time))

woa_report2 = classification_report(y_test2, (y_pred_best_wao > 0.5).astype(int))

print(woa_report2)

8091/8091 [==============================] - 15s 2ms/step - loss: 0.0534 - accuracy: 0.9812
Iteration 1/10, Loss: 0.053361181169748306
8091/8091 [==============================] - 15s 2ms/step - loss: 0.0133 - accuracy: 0.9948
Iteration 2/10, Loss: 0.01326997671276331
8091/8091 [==============================] - 16s 2ms/step - loss: 17.2216 - accuracy: 0.9890
Iteration 3/10, Loss: 17.221593856811523
8091/8091 [==============================] - 16s 2ms/step - loss: 0.0449 - accuracy: 0.9962
Iteration 4/10, Loss: 0.04487629979848862
8091/8091 [==============================] - 16s 2ms/step - loss: 932.6694 - accuracy: 0.6079
Iteration 5/10, Loss: 932.6693725585938
8091/8091 [==============================] - 16s 2ms/step - loss: 0.0013 - accuracy: 0.9999
Iteration 6/10, Loss: 0.0012816726230084896
8091/8091 [==============================] - 16s 2ms/step - loss: 2.6229 - accuracy: 0.9964
Iteration 7/10, Loss: 2.6228785514831543
8091/8091 [==============================] - 19s 2ms/step - 